## Intent detector

In [54]:
user_input = "I completely agree with you!"

import json
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

with open("all_intents.json", 'r') as json_in: # all_intents.json can be found in the task-intent-detector/model_code directory
    data = json.load(json_in)

id_to_intent, intent_to_id = dict(), dict()
for i, intent in enumerate(data):
    id_to_intent[i] = intent
    intent_to_id[intent] = i

model = AutoModelForSequenceClassification.from_pretrained("NOVA-vision-language/task-intent-detector", num_labels=len(data), id2label=id_to_intent, label2id=intent_to_id)
tokenizer = AutoTokenizer.from_pretrained("roberta-base") # you could try 'NOVA-vision-language/task-intent-detector', but I'm not sure I configured it correctly

model_in = tokenizer(user_input, return_tensors='pt')
with torch.no_grad():
    logits = model(**model_in).logits # grab the predictions out of the model's classification head
    predicted_class_id = logits.argmax().item() # grab the index of the highest scoring output
    predicted_intent = model.config.id2label[predicted_class_id] # use the translation table we just created to translate between that id and the actual intent name

## Slot-filling with QA

In [60]:
intent_slot_mapping = {
    "GreetingIntent": ['What is the time of day', 'What is the name of the user?'],  # (1) Greetings
    "IdentifyProcessIntent": ['What are the ingredients?', 'What is the cuisine type?', 'What is the meal type?'],  # (2) Search recipe
    "OutOfScopeIntent": [],  # (3) Out of scope
    "YesIntent": [],  # (4) Yes
    "NoIntent": [],  # (5) No
    "StartStepsIntent": ['What is the recipe?'],  # (6) Start task
    "NextStepIntent": ['Current Step Number'],  # (7) Next
    "StopIntent": ['What is the reason for stopping?'],  # (8) Stop
}


model_name = "deepset/roberta-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

if predicted_intent in intent_slot_mapping:
    print('Intent: ' + predicted_intent + '\n')
    for slot in intent_slot_mapping[predicted_intent]:
        QA_input = {
            'question': slot,
            'context': user_input
        }
        res = nlp(QA_input)
        print('Slot: ' + slot)
        print('Slot Value: ' + str(res) + '\n')
else:
    print("Intent not found in mapping")
    print("Intent: " + predicted_intent)

Intent: YesIntent



In [53]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Meal Type',
    'context': 'Give me a typical Indian lunch recipe that has chicken breasts as an ingredient'
}
res = nlp(QA_input)
res

{'score': 0.005084468983113766,
 'start': 18,
 'end': 62,
 'answer': 'Indian lunch recipe that has chicken breasts'}